In [1]:
from numpy import array
from numpy import asarray
from numpy import zeros
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from pathlib import Path
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [2]:
#TODO: Download the dataset from https://www.kaggle.com/c/word2vec-nlp-tutorial/data
#TODO: Unzip the dataset and place it in the same folder as this notebook
#TODO: Change the path to the dataset below
# dataset_path = Path(os.getcwd()) / 'data' / 'labeledTrainData.tsv'


In [3]:
# Define split size
TRAIN_SPLIT = 100
VAL_SPLIT = 150

# Define file iterator
def file_iterator():
    data_dir = Path('dependency_treebank')
    for data_file in filter(lambda f: os.path.isfile(data_dir/f) and f.endswith('.dp'), os.listdir(data_dir)):
        yield data_dir/data_file

# Create train, val and test set
train_set = []
test_set = []
val_set = []
file_counter = 0

# Iterate over files and perform split
for file in file_iterator():
    file_counter += 1
    if file_counter <= TRAIN_SPLIT:
        train_set.append(pd.read_csv(file, sep="\t", names=['token', 'pos'], usecols=[0, 1], engine='python'))
    elif file_counter <= VAL_SPLIT:
        val_set.append(pd.read_csv(file, sep="\t", names=['token', 'pos'], usecols=[0, 1], engine='python'))
    else:
        test_set.append(pd.read_csv(file, sep="\t", names=['token', 'pos'], usecols=[0, 1], engine='python'))

# Check for correct split
assert len(train_set) == 100
assert len(val_set) == 50
assert len(test_set) == 49

# Change to pandas dataframe
train_frame = pd.concat(train_set)
test_frame = pd.concat(test_set)
val_frame = pd.concat(val_set)

#TODO: Optimize memmory?

# Check for correct transformation
assert sum([e.shape[0] for e in train_set]) == train_frame.shape[0]
assert sum([e.shape[0] for e in test_set]) == test_frame.shape[0]
assert sum([e.shape[0] for e in val_set]) == val_frame.shape[0]

In [4]:
# Tokenize and padd data

def tokenize_and_pad(data_frame):
    tokenizer = Tokenizer() # Define tokenizer
    tokenizer.fit_on_texts(data_frame['token'].values)
    vocab_size = len(tokenizer.word_index) + 1
    encoded_doc = tokenizer.texts_to_sequences(data_frame['token'].values)
    max_length = 1
    padded_docs = pad_sequences(encoded_doc, maxlen=max_length, padding='post')
    return padded_docs, vocab_size, tokenizer

# train set
train_padded_docs, train_vocab_size, train_tokenizer = tokenize_and_pad(train_frame)

# val set
val_padded_docs, val_vocab_size, val_tokenizer = tokenize_and_pad(val_frame)

# test set
test_padded_docs, test_vocab_size, test_tokenizer = tokenize_and_pad(test_frame)

In [5]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


def create_embedding_matrix(tokenizer, vocab_size):
    # load embedding into memory, skip first
    embedding_matrix = zeros((vocab_size, 100))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            # print("Not in Vocab", word)
            pass
    return embedding_matrix

# Encode labels
def encode_labels(data_frame):
    encoder = LabelEncoder()
    encoder.fit(data_frame['pos'].values)
    encoded_Y = encoder.transform(data_frame['pos'].values)
    # convert integers to dummy variables (i.e. one hot encoded) -> labels
    labels = np_utils.to_categorical(encoded_Y)
    return labels, encoder


Loaded 400000 word vectors.


In [26]:
# create the model
def create_model(vocab_size, embedding_matrix, plot_model=False):
    # define model
    model = Sequential()
    e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=1, trainable=False) # define embedding layer
    model.add(e) # add embedding layer to model
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=128), input_shape=(None, 50))) # add bidirectional LSTM layer to model
    model.add(Dense(44, activation='softmax')) # add dense layer to model

    #TODO: Put in train function
    # compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # summarize the model
    print(model.summary())
    if plot_model:
        keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
    return model

# run the model
def train_model(model, padded_docs, labels):
    # fit the model
    model.fit(padded_docs, labels, epochs=20, verbose=1)

    # evaluate the model
    loss, accuracy = model.evaluate(padded_docs, labels, verbose=1)
    print('Accuracy: %f' % (accuracy * 100))

# def inference(model, tokenizer, vocab_size, test_padded_docs, test_frame):
#     # predict the model
#     yhat = model.predictd(test_padded_docs, verbose=1)
#     # map predicted labels to words
#     predicted_labels = []
#     for i in yhat:
#         for word, index in tokenizer.word_index.items():
#             if index == i:
#                 predicted_labels.append(word)
#                 break
#     # map actual labels to words
#     actual_labels = []
#     for i in test_frame['pos'].values:
#         for word, index in tokenizer.word_index.items():
#             if index == i:
#                 actual_labels.append(word)
#                 break
#     # create confusion matrix
#     confusion_matrix = pd.crosstab(pd.Series(actual_labels), pd.Series(predicted_labels), rownames=['Actual'], colnames=['Predicted'])
#     print(confusion_matrix)


In [27]:
# train model
train_embedding_matrix = create_embedding_matrix(train_tokenizer, train_vocab_size)
train_labels, train_encoder = encode_labels(train_frame)
model = create_model(train_vocab_size, train_embedding_matrix)
train_model(model, train_padded_docs, train_labels)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 1, 100)            632700    
                                                                 
 bidirectional_5 (Bidirectio  (None, 256)              234496    
 nal)                                                            
                                                                 
 dense_5 (Dense)             (None, 44)                11308     
                                                                 
Total params: 878,504
Trainable params: 245,804
Non-trainable params: 632,700
_________________________________________________________________
None
Epoch 1/20
1413/1413 [==============================] - 3s 2ms/step - loss: 1.0032 - accuracy: 0.7219
Epoch 2/20
1413/1413 [==============================] - 2s 2ms/step - loss: 0.5852 - accuracy: 0.8138
Epoch 3/20
1413/1413 [=========

In [21]:
# validate model
val_embedding_matrix = create_embedding_matrix(val_tokenizer, val_vocab_size)
val_labels, val_encoder = encode_labels(val_frame)

loss, accuracy = model.evaluate(val_padded_docs, val_labels, verbose=1)

InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/Users/anani/miniforge3/envs/play/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/anani/miniforge3/envs/play/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/anani/miniforge3/envs/play/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/anani/miniforge3/envs/play/lib/python3.9/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/Users/anani/miniforge3/envs/play/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Users/anani/miniforge3/envs/play/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
  

In [20]:
# predict model val
predictions = model.predict(val_padded_docs, verbose=1)

#TODO: compare



879/879 [==============================] - 1s 613us/step
[[1.0907109e-05 2.2433024e-04 2.1703347e-06 ... 2.8716912e-08
  4.9551250e-07 7.0266455e-05]
 [4.2851013e-08 1.0514050e-06 4.9462809e-08 ... 8.0239711e-11
  5.8290900e-11 1.2585730e-07]
 [2.6137733e-03 6.1667468e-02 2.5079817e-06 ... 2.7957765e-07
  1.7772708e-06 5.3486343e-02]
 ...
 [1.1580768e-07 3.0247747e-06 7.4308076e-08 ... 2.9441841e-11
  2.2790166e-10 4.1862222e-07]
 [4.4803182e-05 1.6333588e-04 4.8778595e-05 ... 6.6113500e-08
  4.7637050e-07 6.0173814e-05]
 [2.6137733e-03 6.1667468e-02 2.5079817e-06 ... 2.7957765e-07
  1.7772708e-06 5.3486343e-02]]


TEST AREA

In [29]:
# import numpy as np
# # predict model test
# predictions = model.predict(test_padded_docs, verbose=1)
# # map predicted labels to words
# predicted_labels = []
# for i in predictions:
#     for word, index in test_tokenizer.word_index.items():
#         if index == np.argmax(i):
#             predicted_labels.append(word)
#             break
# # map actual labels to words
# actual_labels = []
# for i in test_frame['pos'].values:
#     for word, index in test_tokenizer.word_index.items():
#         if index == i:
#             actual_labels.append(word)
#             break
# # create confusion matrix
# confusion_matrix = pd.crosstab(pd.Series(actual_labels), pd.Series(predicted_labels), rownames=['Actual'], colnames=['Predicted'])

44

In [34]:
# import numpy as np
# np.argmax(val_labels[0])
# np.argmax(predictions[0])

35